In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Visualizing of confusion matrix
import seaborn as sn
import pandas  as pd
import matplotlib.pyplot as plt
%matplotlib inline

DESIRED_ACCURACY = 0.999
IMG_SIZE = 224

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
            directory = '/Users/neikusc/data/dogs_cats/train/',
            target_size=(IMG_SIZE,IMG_SIZE),
            batch_size = 128,
            class_mode = 'binary'
        )

valid_datagen = ImageDataGenerator(rescale=1/255.)
valid_generator = valid_datagen.flow_from_directory(
            directory = '/Users/neikusc/data/dogs_cats/valid/',  # This is the source directory for training images
            target_size=(IMG_SIZE,IMG_SIZE),         # All images will be resized to IMG_SIZE x IMG_SIZE
            batch_size=32,                           # Flow training images in batches of 32 using train_datagen generator
            class_mode='binary',                     # Since we use binary_crossentropy loss, we need binary labels
        )

Found 19985 images belonging to 2 classes.
Found 5015 images belonging to 2 classes.


In [3]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=2,  
                epochs=5,
                verbose=1,
                callbacks=[callbacks],
                validation_data = valid_generator)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
157/157 [==============================] - 2003s 13s/step - loss: 0.6939 - acc: 0.6323 - val_loss: 0.5801 - val_acc: 0.6921
Epoch 2/5
157/157 [==============================] - 5346s 34s/step - loss: 0.5316 - acc: 0.7366 - val_loss: 0.5176 - val_acc: 0.7484
Epoch 3/5
157/157 [==============================] - 1845s 12s/step - loss: 0.4587 - acc: 0.7833 - val_loss: 0.5159 - val_acc: 0.7623
Epoch 4/5
157/157 [==============================] - 2077s 13s/step - loss: 0.3945 - acc: 0.8208 - val_loss: 0.4536 - val_acc: 0.7882
Epoch 5/5
157/157 [==============================] - 9440s 60s/step - loss: 0.3186 - acc: 0.8597 - val_loss: 0.4634 - val_acc: 0.7940


In [ ]:
# Plots for training and testing process: loss and accuracy
plt.figure(0)
plt.plot(history.history['acc'],'r')
plt.plot(history.history['val_acc'],'g')
plt.xticks(np.arange(0, 11, 2.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])

plt.figure(1)
plt.plot(history.history['loss'],'r')
plt.plot(history.history['val_loss'],'g')
plt.xticks(np.arange(0, 11, 2.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])

plt.show()

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


# Confusion matrix result

from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict(x_test, verbose=2)
y_pred = np.argmax(Y_pred, axis=1)

for ix in range(10):
    print(ix, confusion_matrix(np.argmax(y_test,axis=1),y_pred)[ix].sum())
cm = confusion_matrix(np.argmax(y_test,axis=1),y_pred)
print(cm)

df_cm = pd.DataFrame(cm, range(10),
                  range(10))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 12})# font size
plt.show()